In [0]:
import numpy as np
import os
import keras, warnings
from keras.models import *
from keras.layers import *
from keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
import csv
from tensorflow.python.client import device_lib
warnings.filterwarnings("ignore")
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11468054659582691581
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5042321174428224210
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16302848543798239753
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9099756955270269506
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
for file in os.listdir("/kaggle/input/modelcloud"):
    print(file)

In [0]:
dir_xtrain_images  = "/content/gdrive/My Drive/Cloud/train_images
dir_test_images  = "/content/gdrive/My Drive/Cloud/test_images"
dir_y_train_files = "/content/gdrive/My Drive/Cloud/Final_Segmented_Files"
dir_csv_output_file = "/content/gdrive/My Drive/Cloud/CSV/mysubmissiohn.csv"


In [0]:
from keras.models import load_model
model = load_model("/content/gdrive/My Drive/Cloud/Models/model178.h5")















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
import os
x_test_List = []
for file in os.listdir(dir_test_images):
    #print(file)
    x_test_List.append(dir_test_images+'/'+file)
    
x_test_List.sort()    
print('Total training Images: ' + str(len(x_test_List)))


Total training Images: 3698


In [0]:
def GetEndodedList(y_predicted,c):
    List = []
    
    numberOfRunningPixel = 0
    
    addable = False
    
    for w in range(525):
        
                for h in range(350):
                
                    if(int(y_predicted[h,w]) == c and numberOfRunningPixel >0):
                        addable =True
                        #wasPreviousSame = False
                        buf = numberOfRunningPixel
                        numberOfRunningPixel +=1
                        
                        continue
                        
                        
                    elif(int(y_predicted[h,w]) == c and numberOfRunningPixel ==0 ):      
                        
                        addable =True
                        start = (w*350)+(h+1)
                        
                        numberOfRunningPixel +=1
                        
                        if(start % 350 == 0):
                            start = (w+1)*350
                        
                        List.append(start)
                        
                        #take care running here 
                        
                    if( addable and not (y_predicted[h,w] == c)  ):
                        
                        addable = False
                        
                        List.append(numberOfRunningPixel)
                        
                        numberOfRunningPixel = 0
                            
                             
                        
                        
    
    
    
    return List

In [0]:
def calculate_chunk_size(total_image_number, chunk_size):
    list_of_chunks= []
    n_of_whole_chunk_sizes = int(total_image_number/chunk_size)
    n_of_partial_chunk_sizes = total_image_number%chunk_size
    for i in range(n_of_whole_chunk_sizes):
        list_of_chunks.append(chunk_size)
    if(n_of_partial_chunk_sizes != 0):
        list_of_chunks.append(n_of_partial_chunk_sizes)
    print("List of chunks= " , list_of_chunks)
    print("Length of List of chunks= " , len(list_of_chunks))        
    return list_of_chunks       

In [0]:
import cv2
original_image_width  = 2100
original_image_height = 1400
image_width = 2080
image_height = 1376
channel_size = 3
number_of_classes = 7 # 7 in this project
chunk_size = 1
total_test_image_number = len(x_test_List)  # 5546 in ths project
start = 0;
end = start + chunk_size
list_of_chunks = calculate_chunk_size(total_test_image_number, chunk_size)
batchSize = 2
epoch = 4
needResize = False
beginA = 0

num_testing_image = len(x_test_List)




with open(dir_csv_output_file, 'a+') as myFile:   # a+ . + == create if not exists
              MyCsvWriterObject = csv.writer(myFile)
              MyCsvWriterObject.writerow(['Image_Label','EncodedPixels'])

# channel last with tensorflow backend








for beginA in range(len(list_of_chunks)):
    
    print("starting: " , start, " ", end)
    x_test = np.zeros((list_of_chunks[beginA], image_height, image_width, channel_size),dtype = 'uint8')
    
    
    
    #GET X-TRAIN
    for j in range(start,end):
        print("Creating x_test: " , x_test_List[j].split('/')[6])
        image_data = cv2.imread(x_test_List[j])
        image_data = image_data.astype('uint8')
        image_data = cv2.resize(image_data, (image_width, image_height), interpolation = cv2.INTER_AREA)
        x_test[j-start] =  image_data

    
    
    y_pred = model.predict(x_test)  
    y_predi = np.argmax(y_pred, axis=3)  

###############################3cvs########################################3
    with open(dir_csv_output_file, 'a+') as myFile:   # a+ . + == create if not exists
              MyCsvWriterObject = csv.writer(myFile)
              readCSV = csv.reader(myFile)
              for row in readCSV:
                if(row[0] == None):
                  MyCsvWriterObject.writerow(['Image_Label','EncodedPixels'])
                  break
                else:
                  break
            
          
            
              for kkk in range(len(y_predi)):
                
                    
                  filename = x_test_List[kkk].split('/')[6]
                  
                  print("jjj .  ",filename)
                  
                  
                  y_predicted = y_predi[kkk]
                  
                  for c in range(number_of_classes):
                      if c == 0 or c == 6 or c == 5:
                          continue
                      if(c == 1):
                          classid = 'Fish'
                      elif(c == 2):
                          classid = 'Flower'
                      elif(c == 3):
                          classid = 'Gravel'    
                      elif(c == 4):
                          classid = 'Sugar'    
                      
                      Listofpixels = GetEndodedList(y_predicted,c)
                      
                                  
                                  
                      #Listofpixels=[2,3,4,5,6,7,8]

                      
                      s=''
                      for t in range(len(Listofpixels)):
                          if t == len(Listofpixels)+1:
                              s +=str(Listofpixels[t]) 
                          else: s +=str(Listofpixels[t])+' '
                              

                      MyCsvWriterObject.writerow([filename+'_'+str(classid), s])
                
    
    #kkk+=1





    
    
    
    start = end
    if( beginA < len(list_of_chunks)-1):           
        end = end + list_of_chunks[beginA+1]
    else:
        end = end + list_of_chunks[beginA]
    
  